In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import LawSchoolDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.experiment_interface import run_exp_iter_with_eq_odds

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
DATASET_NAME = 'LawSchoolDataset'
FAIRNESS_INTERVENTION_NAME = 'ROC'
EXPERIMENT_NAME = 'ROC_law_school'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [True]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'law_school_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
    # 'session_uuid': '73b7936a-14e6-4b86-a927-bf1cd81477d3',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  73b7936a-14e6-4b86-a927-bf1cd81477d3


## Initialize custom objects

In [9]:
data_loader = LawSchoolDataset()
data_loader.X_data.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race
0,6.0,8.0,35.0,4.0,0.28,0.81,1.0,2.0,1.0,4.0,White
1,6.0,7.0,41.0,2.9,0.30,0.33,1.0,5.0,0.0,3.0,White
2,4.0,4.0,36.0,3.3,-0.38,-0.46,1.0,4.0,0.0,3.0,White
3,10.0,8.0,34.0,2.5,1.91,0.77,1.0,3.0,1.0,1.0,White
4,6.0,6.0,34.0,2.8,0.10,0.13,1.0,3.0,1.0,3.0,White


In [10]:
data_loader.X_data.shape

(3000, 11)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# tuned_params_filenames = [
#     'tuning_results_Law_School_20240104__104605.csv'
# ]
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
#                                       FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [12]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [13]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          with_tuning=True,
                          # with_tuning=False,
                          # tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

2024-01-05 17:59:58 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 100,
 'session_uuid': '73b7936a-14e6-4b86-a927-bf1cd81477d3'}

data_loader.X_data.head(20)


,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race,male&race_binary
0,6.0,8.0,35.0,4.0,0.28,0.81,1.0,2.0,1.0,4.0,White,1
1,6.0,7.0,41.0,2.9,0.30,0.33,1.0,5.0,0.0,3.0,White,1
2,4.0,4.0,36.0,3.3,-0.38,-0.46,1.0,4.0,0.0,3.0,White,1
3,10.0,8.0,34.0,2.5,1.91,0.77,1.0,3.0,1.0,1.0,White,1
4,6.0,6.0,34.0,2.8,0.10,0.13,1.0,3.0,1.0,3.0,White,1
5,9.0,8.0,37.0,3.0,0.89,0.55,1.0,4.0,0.0,3.0,White,1
6,7.0,8.0,34.0,3.5,0.47,0.77,1.0,3.0,0.0,3.0,White,1
7,3.0,4.0,34.0,3.4,-0.43,-0.37,1.0,4.0,1.0,3.0,White,1
8,6.0,2.0,35.0,3.5,0.12,-0.87,1.0,4.0,1.0,6.0,White,1
9,3.0,3.0,31.0,3.4,-0.46,-0.59,1.0,4.0,0.0,3.0,White,1


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 17:59:58 experiment_interface.py INFO    : The dataset is preprocessed


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__fulltime_1.0', 'cat__fulltime_2.0', 'cat__fam_inc_1.0',
       'cat__fam_inc_2.0', 'cat__fam_inc_3.0', 'cat__fam_inc_4.0',
       'cat__fam_inc_5.0', 'cat__tier_1.0', 'cat__tier_2.0', 'cat__tier_3.0',
       'cat__tier_4.0', 'cat__tier_5.0', 'cat__tier_6.0', 'num__decile1b',
       'num__decile3', 'num__lsat', 'num__ugpa', 'num__zfygpa', 'num__zgpa',
       'male&race_binary'],
      dtype='object')
Top indexes of an X_test in the current base flow dataset:  Int64Index([1273, 1380,  356, 1920,  561,  105,  999, 2563,  584,   31,  610,
            1856, 2740, 1804, 2691, 1292, 2305,  221,  291, 1290],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([1273, 1380,  356, 1920,  561,  105,  999, 2563,  584,   31,  610,
            1856, 2740, 1804, 2691, 1292, 2305,  221,  291, 1290],
           dtype='int64')
2024/01/05, 17:59:58: Tunin

2024-01-05 17:59:59 experiment_interface.py INFO    : Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 3

In [23]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [2]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 4

In [25]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [3]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 5

In [27]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [4]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 6

In [29]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [5]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)